In [15]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {"movie":"Avatar", 
     "info":"Title: Avatar(2009)\nDirector: James Cameron\nMain Cast: Sam Worthington, Zoe Saldaña, Sigourney Weaver\nBudget: $237 million\nBox Office: $2.92 billion\nGenre: Science Fiction, Action, Adventure\nSynopsis: Set on the lush alien world of Pandora, a paraplegic marine is dispatched to infiltrate the native Na'vi tribe, but becomes torn between following his orders and protecting the world he feels is his home.\n"},
    {"movie":"Avengers Endgame", 
    "info":"Title: Avengers: Endgame(2019)\nDirector: Anthony and Joe Russo\nMain Cast: Robert Downey Jr., Chris Evans, Scarlett Johansson\nBudget: $356 million\nBox Office: $2.80 billion\nGenre: Superhero, Action, Sci-Fi\nSynopsis: The Avengers assemble once more to undo the catastrophic events caused by Thanos in their previous battle, leading to a final showdown to save the universe.\n"},
    # {"movie":"Avatar The Way of Water", 
    # "info":"Title: Avatar: The Way of Water (2022)\nDirector: James Cameron\nMain Cast: Sam Worthington, Zoe Saldaña, Sigourney Weaver\nBudget: $350 million\nBox Office: $2.32 billion\nGenre: Science Fiction, Action, Adventure\nSynopsis: Jake and Neytiri have formed a family, but when an old threat resurfaces, they are forced to leave their home and explore the regions of Pandora, including the oceans."},
    # {"movie":"Titanic", 
    # "info":"Title: Titanic (1997)\nDirector: James Cameron\nMain Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane\nBudget: $200 million\nBox Office: $2.23 billion\nGenre: Romance, Drama\nSynopsis: A young couple from different social backgrounds fall in love aboard the ill-fated R.M.S. Titanic, leading to a tragic and unforgettable love story."},
    # {"movie":"Star Wars: The Force Awakens",
    #  "info":"Title: Star Wars: The Force Awakens (2015)\nDirector: J.J. Abrams\nMain Cast: Daisy Ridley, John Boyega, Harrison Ford\nBudget: $245 million\nBox Office: $2.06 billion\nGenre: Science Fiction, Action, Adventure\nSynopsis: A new heroine emerges in the galaxy as the Resistance faces the First Order, leading to the return of familiar faces and the discovery of new heroes."}
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Tell me about {movie}."),
    ("ai","{info}")
])

fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a movie expert"),
    fewshot_example_prompt,
    ("human","Tell me about {movie}.")
])

print(final_prompt.format(movie="Millenium"),"\n\n") #to see if the prompt is correct

chain = final_prompt | chat
millenium_info = chain.invoke("The Girl with the Dragon Tattoo (the American adaptation)")


System: you are a movie expert
Human: Tell me about Avatar.
AI: Title: Avatar(2009)
Director: James Cameron
Main Cast: Sam Worthington, Zoe Saldaña, Sigourney Weaver
Budget: $237 million
Box Office: $2.92 billion
Genre: Science Fiction, Action, Adventure
Synopsis: Set on the lush alien world of Pandora, a paraplegic marine is dispatched to infiltrate the native Na'vi tribe, but becomes torn between following his orders and protecting the world he feels is his home.

Human: Tell me about Avengers Endgame.
AI: Title: Avengers: Endgame(2019)
Director: Anthony and Joe Russo
Main Cast: Robert Downey Jr., Chris Evans, Scarlett Johansson
Budget: $356 million
Box Office: $2.80 billion
Genre: Superhero, Action, Sci-Fi
Synopsis: The Avengers assemble once more to undo the catastrophic events caused by Thanos in their previous battle, leading to a final showdown to save the universe.

Human: Tell me about Millenium. 


Title: The Girl with the Dragon Tattoo(2011)
Director: David Fincher
Main Cast

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

#examples
examples = [
    {"movie":"Titanic", "emoji":"🍝💼🔫"},
    {"movie":"Star Wars", "emoji":"🌌⚔️👽"},
    {"movie":"The Lord of the Rings", "emoji":"🧙‍♂️⚔️💍"},
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Express {movie} in three emojis."),
    ("ai","{emoji}")
])
fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples
)

#memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    memory_key = "history",
    return_messages=True
)
def load_memory(_):
    print(_)
    return memory.load_memory_variables({})["history"]

#prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a movie and emoji expert."),
    fewshot_example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human","Tell me about {movie}.")
])


#LCEL
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input":movie}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("Harry Potter")
invoke_chain("50 shades of grey")


/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_46998/2826024641.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


{'movie': 'Harry Potter'}
content='Harry Potter is a popular fantasy book series written by J.K. Rowling that follows the adventures of a young wizard named Harry Potter and his friends Hermione Granger and Ron Weasley as they attend Hogwarts School of Witchcraft and Wizardry. The series explores themes of friendship, courage, and the battle between good and evil, with Harry facing the dark wizard Lord Voldemort. The books have been adapted into a successful film series and have a dedicated fan base around the world.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 104, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BPwaVybzxPd1ZWfPcqokREic3T6Nz', 'finish_reason': 's

In [ ]:
#일단 example 테스트해보자
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

#examples
examples = [
    {"movie":"Titanic", "emoji":"🍝💼🔫"},
    {"movie":"Star Wars", "emoji":"🌌⚔️👽"},
    {"movie":"The Lord of the Rings", "emoji":"🧙‍♂️⚔️💍"},
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Express {movie} in three emojis."),
    ("ai","{emoji}")
])
fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples
)

#prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a movie and emoji expert."),
    fewshot_example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human","Tell me about {movie}.")
])


#LCEL
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input":movie}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("Harry Potter")
invoke_chain("50 shades of grey")
